#  Make model ( trainig, testig)

In [21]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
from sklearn.utils import check_random_state
import joblib


In [22]:
def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    
    #mfcc feature
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta mfcc feature 1st order and 2nd order
    delta1_mfcc = librosa.feature.delta(mfcc, order=1)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1_mfcc, delta2_mfcc], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    print('data_dir: ', data_dir)
    print(f'mfcc.shape: {np.array(mfcc).shape}')
    return mfcc

def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=100, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans  

def save_mod(name, model):
  joblib.dump(model, f"{name}.pkl")
  print(f"Save model {name} success")
    
    
class_names = [ "yte", "dich", "hoc", "cachly", "nha", "test_nha", "test_hoc", "test_cachly", "test_dich" , "test_yte" ]
dataset = {}
for cname in class_names:
    print(f"Load {cname} dataset")
    dataset[cname] = get_class_data(os.path.join("../full_data", cname))

# Get all vectors in the datasets
all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items() if(not k.startswith("test"))], axis=0)
print("vectors", all_vectors.shape)

# Run K-Means algorithm to get clusters
kmeans = clustering(all_vectors, 20)
print("centers", kmeans.cluster_centers_.shape)
joblib.dump(kmeans, f"kmeans_1.pkl")
print(f"Save kmeans_1 success")


test_datas = {}
num_state = {}


startprob = {}

Load yte dataset
data_dir:  ../full_data/yte
mfcc.shape: (67,)
Load dich dataset
data_dir:  ../full_data/dich
mfcc.shape: (60,)
Load hoc dataset
data_dir:  ../full_data/hoc
mfcc.shape: (76,)
Load cachly dataset
data_dir:  ../full_data/cachly
mfcc.shape: (75,)
Load nha dataset
data_dir:  ../full_data/nha
mfcc.shape: (69,)
Load test_nha dataset
data_dir:  ../full_data/test_nha
mfcc.shape: (23,)
Load test_hoc dataset
data_dir:  ../full_data/test_hoc
mfcc.shape: (23,)
Load test_cachly dataset
data_dir:  ../full_data/test_cachly
mfcc.shape: (23,)
Load test_dich dataset
data_dir:  ../full_data/test_dich
mfcc.shape: (21,)
Load test_yte dataset
data_dir:  ../full_data/test_yte
mfcc.shape: (20,)
vectors (33614, 36)
centers (20, 36)
centers (20, 36)
Save kmeans_1 success


In [23]:
startprob['dich'] = np.array([0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ])
startprob['hoc'] = np.array([0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ])
startprob['yte'] = np.array([0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ])
startprob['cachly'] = np.array([0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,  0.  , 0.  , 0.  , 0.  , 0.  ])
# |ŋ|-|ʷə|-|j|
startprob['nha'] = np.array([0.7 , 0.2 , 0.05, 0.05, 0.  , 0.   ])

num_state['dich'] = 9
num_state['hoc'] = 9
num_state['yte'] = 10
num_state['cachly'] = 16
num_state['nha'] = 6

transmat = {}
transmat['dich'] = np.array(  [[0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.1 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

transmat['hoc'] = np.array(  [[0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05, 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.05, 0.05],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.2 , 0.1 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

transmat['yte'] = np.array([[0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.1 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.2 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
                           [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

transmat['cachly'] = np.array([[0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,    0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.05, 0.05],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.1 , 0.1 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.1 , 0.2 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

transmat['nha'] = np.array([[0.7, 0.2, 0.1, 0. , 0. , 0. ],
                           [0. , 0.7, 0.2, 0.1, 0. , 0. ],
                           [0. , 0. , 0.7, 0.2, 0.1, 0. ],
                           [0. , 0. , 0. , 0.7, 0.2, 0.1],
                           [0. , 0. , 0. , 0. , 0.7, 0.3],
                           [0. , 0. , 0. , 0. , 0. , 1. ]])

train_class_name = [ "yte", "dich", "hoc", "cachly", "nha"]

iteration_ = 10

while(iteration_>0):
    models = {}
    train_datas = {}
    iteration_ -= 1
    for cname in train_class_name :

        #Clustering 
        train_datas[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])

        #Initialization
        hmm = hmmlearn.hmm.MultinomialHMM(
            n_components=num_state[cname], random_state= iteration_ , n_iter=1000, verbose=True,
            params='te',
            init_params='e'
        )
        
        hmm.startprob_= startprob[cname]
        hmm.n_features = 20
        hmm.transmat_= transmat[cname]

        #Fiting data and train
        X = np.concatenate(train_datas[cname])
        lengths = list([len(x) for x in train_datas[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
        
        # save model
        joblib.dump(models[cname], f"{cname}_iter{iteration_}.pkl")
        print(f"Save model {cname}_iter{iteration_} successfully")

    print(f"\nIter {iteration_} : Finish training\n")

        
    testing_set = {}
    for k, v in train_datas.items():
        testing_set[k] = list([kmeans.predict(v).reshape(-1,1) for v in dataset["test_"+k]])

    num_true = {}
    num_false = {}

    #Initialization:
    for name in train_class_name:
        num_true[name] = 0
        num_false[name] = 0

    #Testing:
    for true_cname in train_class_name:
        print(f"Testing word {true_cname}:")
        for O in testing_set[true_cname]:
            scores = {cname : model.score(O, [len(O)]) for cname, model in models.items()}
            pred_name = max(scores, key=lambda key: scores[key])                
            if(pred_name == true_cname):
                num_true[true_cname] += 1
            else:
                num_false[true_cname] += 1            
    print("Test Result: ")
    for name in train_class_name:
        acc =  100*num_true[name]/(num_true[name]+num_false[name])
        print(f"Acc on word \"{name}\": {acc}")
        print(f"Correct Rate: {num_true[name]}\\{num_true[name]+num_false[name]} ")
        print()
    sum_true = sum(num_true.values())
    sum_false = sum(num_false.values())
    print(f"Overall Acc: {sum_true*100 / (sum_true+sum_false)}")
    
    print(f"\nIter {iteration_} : Finish testing\n******************************************************\n\n")            

training class yte
(2954, 1) [43, 34, 42, 35, 44, 29, 47, 28, 40, 50, 38, 40, 39, 35, 37, 54, 29, 37, 54, 53, 31, 41, 61, 37, 33, 50, 42, 48, 62, 39, 44, 34, 31, 42, 49, 77, 45, 44, 61, 66, 31, 47, 49, 35, 79, 34, 39, 50, 41, 39, 44, 68, 32, 38, 36, 34, 53, 44, 53, 38, 49, 60, 43, 41, 48, 41, 43] 67
Have 20 features
Set n_features success


         1       -8431.3428             +nan
         2       -6608.1853       +1823.1575
         3       -6151.9797        +456.2056
         4       -5813.2980        +338.6817
         5       -5537.7866        +275.5113
         6       -5385.1521        +152.6345
         7       -5280.5689        +104.5832
         8       -5218.9916         +61.5774
         9       -5177.4389         +41.5527
        10       -5158.0625         +19.3764
        11       -5145.2246         +12.8379
        12       -5129.7458         +15.4788
        13       -5121.2106          +8.5352
        14       -5116.1634          +5.0472
        15       -5111.0902          +5.0732
        16       -5101.9677          +9.1225
        17       -5082.0031         +19.9646
        18       -5066.5880         +15.4151
        19       -5056.9957          +9.5923
        20       -5048.7786          +8.2171
        21       -5040.0432          +8.7354
        22       -5034.0797          +5.9635
        23

Save model yte_iter9 successfully
training class dich
(5099, 1) [71, 87, 90, 67, 97, 103, 103, 92, 67, 72, 78, 83, 88, 95, 63, 78, 92, 98, 98, 82, 85, 89, 84, 87, 61, 88, 91, 98, 97, 85, 84, 72, 79, 92, 92, 65, 79, 88, 84, 91, 92, 81, 98, 98, 78, 101, 66, 73, 91, 88, 88, 85, 101, 89, 62, 84, 88, 70, 83, 88] 60
Have 20 features
Set n_features success


         1      -14418.5128             +nan
         2       -6051.3146       +8367.1982
         3       -5106.3204        +944.9942
         4       -4759.0400        +347.2803
         5       -4619.8614        +139.1786
         6       -4527.2219         +92.6395
         7       -4485.4843         +41.7376
         8       -4476.6910          +8.7933
         9       -4473.1974          +3.4936
        10       -4471.3766          +1.8208
        11       -4470.4609          +0.9157
        12       -4469.9417          +0.5192
        13       -4469.6397          +0.3020
        14       -4469.4625          +0.1772
        15       -4469.3584          +0.1040
        16       -4469.2975          +0.0609
        17       -4469.2616          +0.0359
        18       -4469.2401          +0.0216
        19       -4469.2265          +0.0135
        20       -4469.2176          +0.0090


Save model dich_iter9 successfully
training class hoc
(10532, 1) [118, 107, 140, 154, 129, 101, 148, 153, 145, 155, 108, 130, 153, 153, 135, 152, 155, 168, 125, 137, 124, 173, 173, 112, 122, 141, 155, 130, 148, 153, 163, 163, 133, 120, 125, 155, 144, 161, 143, 153, 137, 119, 158, 116, 155, 130, 140, 148, 141, 153, 104, 145, 127, 145, 126, 126, 120, 120, 125, 132, 135, 129, 134, 161, 148, 163, 133, 124, 124, 130, 153, 127, 158, 130, 155, 104] 76
Have 20 features
Set n_features success


...

Save model hoc_iter0 successfully
training class cachly
(4002, 1) [82, 50, 56, 33, 61, 43, 65, 69, 67, 76, 49, 31, 61, 41, 90, 33, 39, 61, 49, 37, 61, 71, 38, 63, 51, 39, 68, 32, 65, 49, 34, 64, 36, 50, 43, 55, 89, 60, 59, 56, 70, 58, 61, 37, 30, 51, 68, 51, 60, 63, 59, 28, 61, 59, 28, 60, 29, 59, 64, 57, 64, 33, 89, 46, 35, 59, 56, 89, 35, 35, 59, 50, 54, 34, 35] 75
Have 20 features
Set n_features success


         1      -11586.5822             +nan
         2       -7837.5355       +3749.0467
         3       -6959.5878        +877.9478
         4       -6599.5959        +359.9919
         5       -6468.3894        +131.2064
         6       -6397.3936         +70.9958
         7       -6335.2649         +62.1287
         8       -6282.7807         +52.4842
         9       -6237.4276         +45.3531
        10       -6216.8799         +20.5477
        11       -6205.4086         +11.4713
        12       -6197.1763          +8.2323
        13       -6192.4084          +4.7679
        14       -6189.3248          +3.0837
        15       -6187.1768          +2.1479
        16       -6185.3977          +1.7792
        17       -6183.6528          +1.7449
        18       -6182.0468          +1.6060
        19       -6180.8709          +1.1759
        20       -6179.9211          +0.9498
        21       -6179.0942          +0.8269
        22       -6178.3597          +0.7346
        23

Save model cachly_iter0 successfully
training class nha
(11027, 1) [153, 159, 145, 150, 137, 159, 154, 154, 140, 181, 156, 194, 132, 225, 148, 166, 140, 210, 163, 145, 130, 143, 143, 158, 153, 186, 137, 192, 179, 186, 143, 153, 135, 156, 137, 159, 166, 163, 156, 140, 158, 184, 152, 142, 147, 155, 173, 179, 147, 143, 163, 143, 189, 145, 122, 204, 140, 137, 161, 148, 158, 156, 171, 148, 220, 217, 147, 207, 145] 69
Have 20 features
Set n_features success


         1      -33190.8020             +nan
         2      -15122.4250      +18068.3770
         3      -12723.8589       +2398.5661
         4      -12559.9404        +163.9186
         5      -12469.8380         +90.1024
         6      -12411.8179         +58.0201
         7      -12375.1732         +36.6447
         8      -12344.9138         +30.2594
         9      -12328.6958         +16.2180
        10      -12320.4044          +8.2914
        11      -12316.6773          +3.7271
        12      -12314.4616          +2.2157
        13      -12312.1973          +2.2644
        14      -12309.8131          +2.3842
        15      -12308.4776          +1.3355
        16      -12307.9212          +0.5564
        17      -12307.3964          +0.5248
        18      -12306.2059          +1.1905
        19      -12305.1641          +1.0417
        20      -12304.6491          +0.5151
        21      -12304.3762          +0.2729
        22      -12304.2184          +0.1578
        23

Save model nha_iter0 successfully

Iter 0 : Finish training

Testing word yte:
Testing word dich:
Testing word hoc:
Testing word cachly:
Testing word nha:
Test Result: 
Acc on word "yte": 80.0
Correct Rate: 16\20 

Acc on word "dich": 100.0
Correct Rate: 21\21 

Acc on word "hoc": 100.0
Correct Rate: 23\23 

Acc on word "cachly": 95.65217391304348
Correct Rate: 22\23 

Acc on word "nha": 100.0
Correct Rate: 23\23 

Overall Acc: 95.45454545454545

Iter 0 : Finish testing
******************************************************




In [4]:
np.array(models['nha'].n_features)

array(20)

In [5]:
np.array(models['yte'].n_features)

array(20)

In [6]:
np.array(models['dich'].emissionprob_).shape

(9, 20)

In [7]:
np.array(models['cachly'].emissionprob_).shape

(16, 20)

In [8]:
np.array(models['hoc'].emissionprob_).shape

(9, 20)

# load saved models


In [30]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
from sklearn.utils import check_random_state
import joblib

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    
    #mfcc feature
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta mfcc feature 1st order and 2nd order
    delta1_mfcc = librosa.feature.delta(mfcc, order=1)
    delta2_mfcc = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1_mfcc, delta2_mfcc], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    print('data_dir: ', data_dir)
    print(f'mfcc.shape: {np.array(mfcc).shape}')
    return mfcc


class_names = [ "dich","yte", "hoc", "cachly","nha" ]

#Test load model
kmeans = joblib.load("kmeans_1.pkl")
check_models = {}


iteration_ = 10
while iteration_ > 0:
    iteration_ -= 1
    for cname in class_names:
      print(cname+'_iter'+str(iteration_)) 
      check_models[cname+'_iter'+str(iteration_)] = joblib.load(f"{cname}_iter{iteration_}.pkl") 

        
        
        
dataset = {}
print(f"Load mic dataset")
# mic_classes = ['validation_dich','validation_cachly', 'validation_yte', 'validation_tien', 'validation_nguoi']
# mic_classes = ['hang_dich','hang_cachly', 'hang_yte', 'hang_hoc', 'hang_nguoi']
mic_classes = ['vananh_dich','hang_cachly', 'hang_yte', 'hang_hoc', 'hang_nha']
# mic_classes = ['qa_dich','qa_cachly', "qa_yte", 'qa_hoc', 'qa_nha']
for c_name in mic_classes:
    print(f"Loading {c_name} data")
    dataset[c_name] = get_class_data(os.path.join("../full_data", c_name)) 
        

dich_iter9
yte_iter9
hoc_iter9
cachly_iter9
nha_iter9
dich_iter8
yte_iter8
hoc_iter8
cachly_iter8
nha_iter8
dich_iter7
yte_iter7
hoc_iter7
cachly_iter7
nha_iter7
dich_iter6
yte_iter6
hoc_iter6
cachly_iter6
nha_iter6
dich_iter5
yte_iter5
hoc_iter5
cachly_iter5
nha_iter5
dich_iter4
yte_iter4
hoc_iter4
cachly_iter4
nha_iter4
dich_iter3
yte_iter3
hoc_iter3
cachly_iter3
nha_iter3
dich_iter2
yte_iter2
hoc_iter2
cachly_iter2
nha_iter2
dich_iter1
yte_iter1
hoc_iter1
cachly_iter1
nha_iter1
dich_iter0
yte_iter0
hoc_iter0
cachly_iter0
nha_iter0
Load mic dataset
Loading vananh_dich data
data_dir:  ../full_data/vananh_dich
mfcc.shape: (10,)
Loading hang_cachly data
data_dir:  ../full_data/hang_cachly
mfcc.shape: (6,)
Loading hang_yte data
data_dir:  ../full_data/hang_yte
mfcc.shape: (7,)
Loading hang_hoc data
data_dir:  ../full_data/hang_hoc
mfcc.shape: (7,)
Loading hang_nha data
data_dir:  ../full_data/hang_nha
mfcc.shape: (7,)


#  Most frequency

In [31]:
    
total_true = {}
total_false = {}
mic_set = {}
#Clustering 
for c_name in mic_classes:
    mic_set[c_name] = dataset[c_name]
    total_true[c_name] = 0
    total_false[c_name] = 0

for c_name in mic_classes:
    print('***\n', c_name)  
    pred_list = []
    for i in range(len(mic_set[c_name])):        
        O =  kmeans.predict(mic_set[c_name][i]).reshape(-1,1)  
        
        for iteration_ in range(10):
            tem_models = { f"yte_iter{str(iteration_)}": check_models['yte_iter'+str(iteration_)], f"dich_iter{str(iteration_)}": check_models['dich_iter'+str(iteration_)], f"hoc_iter{str(iteration_)}": check_models['hoc_iter'+str(iteration_)],f"cachly_iter{str(iteration_)}": check_models['cachly_iter'+str(iteration_)],f"nha_iter{str(iteration_)}": check_models['nha_iter'+str(iteration_)]  }
            
            scores = {cname : model.score(O, [len(O)]) for cname, model in tem_models.items()}
            
            pred_name = max(scores, key=lambda key: scores[key])   
            pred_name = pred_name[:pred_name.find('_')]
            pred_list.append(pred_name)
        
        word_counter = {'dich' : 0, 'yte':0, 'hoc': 0, 'nha':0, 'cachly': 0}
        for word in pred_list:
            word_counter[word] += 1
        
        print("word counter: ", word_counter)
        popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
        
        final_pred = popular_words[:1][0]
        
      
        print('final pred : ', final_pred , '\n', '*',c_name[c_name.rfind('_')+1:],'*')
        if( final_pred == c_name[c_name.rfind('_')+1:].strip() ):
            total_true[c_name] += 1
            print("Prediction is correct ! ")
        else:
            total_false[c_name] += 1
            print("False prediction !")

 
print("Result on validation data:")
lst_acc = []
for name in mic_classes:
    acc =  100*total_true[name]/(total_true[name]+total_false[name])
    lst_acc.append(acc)
    print(f"Acc on word \"{name}\": {acc}")
    print(f"Correct Rate: {total_true[name]}\\{total_true[name]+total_false[name]} ")
    print()
sum_true = sum(total_true.values())
sum_false = sum(total_false.values())
print(f"Overall Acc: {sum_true*100 / (sum_true+sum_false)}")
print(f"Average Acc: {sum(lst_acc)/len(lst_acc)}")    
    


***
 vananh_dich
word counter:  {'dich': 0, 'yte': 7, 'hoc': 0, 'nha': 0, 'cachly': 3}
final pred :  yte 
 * dich *
False prediction !
word counter:  {'dich': 9, 'yte': 8, 'hoc': 0, 'nha': 0, 'cachly': 3}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 18, 'yte': 9, 'hoc': 0, 'nha': 0, 'cachly': 3}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 18, 'yte': 17, 'hoc': 0, 'nha': 0, 'cachly': 5}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 27, 'yte': 18, 'hoc': 0, 'nha': 0, 'cachly': 5}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 34, 'yte': 21, 'hoc': 0, 'nha': 0, 'cachly': 5}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 44, 'yte': 21, 'hoc': 0, 'nha': 0, 'cachly': 5}
final pred :  dich 
 * dich *
Prediction is correct ! 
word counter:  {'dich': 44, 'yte': 31, 'hoc': 0, 'nha': 0, 'cachly': 5}
final pred :  dich 
 * dich *
Pred

In [11]:
mic_classes

['hang_dich', 'hang_cachly', 'hang_yte', 'hang_hoc', 'hang_nha']

# Max score ( không hiệu quả bằng Most frequency )

In [26]:
    
total_true = {}
total_false = {}
mic_set = {}
#Clustering 
for c_name in mic_classes:
    mic_set[c_name] = dataset[c_name]
    total_true[c_name] = 0
    total_false[c_name] = 0

for c_name in mic_classes:
    print('***\n', c_name)    
    for i in range(len(mic_set[c_name])):        
        O =  kmeans.predict(mic_set[c_name][i]).reshape(-1,1)        
        scores = {cname : model.score(O, [len(O)]) for cname, model in check_models.items()}
        
        pred_name = max(scores, key=lambda key: scores[key])   
#         print('pred_name: ', pred_name[:pred_name.find('_')], '\n', '*',c_name[c_name.rfind('_')+1:],'*')
        if( pred_name[:pred_name.find('_')] == c_name[c_name.rfind('_')+1:].strip() ):
            total_true[c_name] += 1
#             print("Prediction is correct ! ")
        else:
            total_false[c_name] += 1
#             print("False prediction !")

 
print("Result on validation data:")
lst_acc = []
for name in mic_classes:
    acc =  100*total_true[name]/(total_true[name]+total_false[name])
    lst_acc.append(acc)
    print(f"Acc on word \"{name}\": {acc}")
    print(f"Correct Rate: {total_true[name]}\\{total_true[name]+total_false[name]} ")
    print()
sum_true = sum(total_true.values())
sum_false = sum(total_false.values())
print(f"Overall Acc: {sum_true*100 / (sum_true+sum_false)}")
print(f"Average Acc: {sum(lst_acc)/len(lst_acc)}")    
    


***
 qa_dich
***
 qa_cachly
***
 qa_yte
***
 qa_hoc
***
 qa_nha
Result on validation data:
Acc on word "qa_dich": 71.42857142857143
Correct Rate: 5\7 

Acc on word "qa_cachly": 57.142857142857146
Correct Rate: 4\7 

Acc on word "qa_yte": 57.142857142857146
Correct Rate: 4\7 

Acc on word "qa_hoc": 28.571428571428573
Correct Rate: 2\7 

Acc on word "qa_nha": 85.71428571428571
Correct Rate: 6\7 

Overall Acc: 60.0
Average Acc: 60.0
